# Class Grammar

In [2]:
from pprint import pprint
from collections import Counter
from PYEVALB import scorer
from PYEVALB import parser
import operator
import pickle
import copy
#import progressbar
import numpy as np

In [9]:
#Variables à parser par la suite
filepath='HW2/sequoia-corpus+fct.mrg_strict' #'sequoia.txt'

#Main
with open(filepath, 'r') as f:
        corpus = [line.strip('\n') for line in f]
        
print(len(corpus))
print(corpus[3])
        
corpus_train=corpus[:2479]
corpus_test=corpus[2789:]
corpus_dev=corpus[2479:2789]

print(len(corpus_train), len(corpus_dev), len(corpus_test), len(corpus_train)+len(corpus_dev)+len(corpus_test))

3099
( (SENT (ADV Ensuite) (PONCT ,) (VN (V fut) (VPP installée)) (NP-SUJ (DET une) (ADJ autre) (NC forge)) (PP-P_OBJ (P à) (NP (DET la) (NPP Vacquerie))) (PONCT ,) (PP-P_OBJ (P à) (NP (DET l') (NC emplacement) (ADV aujourd'_hui) (PP (P de) (NP (NPP Cora))))) (PONCT .)))
2479 310 310 3099


# Class Corpus 

In [11]:
class CorpusManager:
    """grammar class"""

    #polyglot-fr.pkl
    def __init__(self,corpus):
        self.corpus = corpus
        self.grammar = {}
        self.lexic = {}
        self.unigram = {} 
        self.bigram ={}
        self.sentences=[]
        self.dic_nodes={}  
        self.term_node=[]
        self.mat_grammar=[]

        
    
    ###################Tools########################    
   
    
    #Erase the part after the dash in POS
    def erase_dash(self):
        for i,sent in enumerate(self.corpus):
            temp = sent.split('(')
            for j,k in enumerate(temp):
                if not len(k.split())==0:
                    temp[j] =  k.split()[0].split('-')[0]+' '+' '.join(k.split()[1:])
            self.corpus[i]='('.join(temp)

                  
    #Return  a list of words from a bracket sentence
    def bracket_to_word(self,sent):
        sent = sent[7:]   
        words=[]
        tmp=sent.split(")")
        for t in tmp:
            try:
                words.append(t.split()[-1])
            except:
                pass
        return words
     
    #build lexic
    def build_lexic(self):
        lexic={}

        for sent in self.corpus:
            st=sent.split("(")
            for i in range(len(st)):
                if ")" in st[i]:
                    l=st[i].replace(")","")
                    word=l.split()[1]

                    tp=l.split()[0]
                    tp=tp.split("-")[0]
                    if word not in lexic.keys():
                        lexic[word] = []
                    lexic[word].append(tp)

        for key in lexic.keys():
                count=Counter(lexic[key])
                lexic[key]=dict(count)

        for key in lexic.keys():
            count=lexic[key]
            total = float(sum(count.values()))
            for k in count:
                count[k] /= total
        self.lexic=lexic
    
    #build bigram values
    def build_bigram(self):
        
        bigrams={}

        for u,sentence in enumerate(self.corpus):
            sentence=self.bracket_to_word(sentence)
            sentence=np.append(sentence,"#e")

            # "#s" and "#e" indicate the starting state and the end state
            sentence=np.insert(sentence,0,"#s")
            for i in range(len(sentence)-1):
                seq = sentence[i]
            
                if  seq not in bigrams.keys():
                    bigrams[seq] = []
                next_chain=sentence[i+1]
                
                if "#e" in next_chain:
                    bigrams[seq].append("#e")
                else:
                    bigrams[seq].append(next_chain)
    
        #count occurences
        for key in bigrams.keys():
            bigrams[key]=Counter(bigrams[key])

        for key in bigrams.keys():
            count=bigrams[key]
            total = float(sum(count.values()))
            for k in count:
                count[k] /= total
            bigrams[key]=dict(count)
        self.bigram = bigrams
         
    # build unigram values
    def build_unigram(self):
        dic_uni={}
        for sent in self.corpus:
            s=self.bracket_to_word(sent)
            for w in s:
                if w not in dic_uni.keys():
                    dic_uni[w]=1
                else:
                    dic_uni[w]+=1
        
       
        total = float(sum(dic_uni.values()))
        for k,v in dic_uni.items():
            dic_uni[k] /= total
        self.unigram = dic_uni
            
    #in order to evaluate on the test
    def build_sentences(self):
        for s in self.corpus:
            self.sentences.append(self.bracket_to_word(s))
    
    #create a dictionary of indices in order to make the CYK algorithm faster
    def build_nodes(self):
        list_nodes=[]
        for k in self.grammar.keys():
            list_nodes.append(k)
            for kb in self.grammar[k].keys():
                r=kb.split("_")
                list_nodes.append(r[0])
                if len(r)==2:
                    list_nodes.append(r[1])
                    
        list_nodes=list(set(list_nodes))
        self.dic_nodes={n:i for i,n in enumerate(list_nodes)}
          

    ###################Build PCFG########################
    
    #build a tree from a bracket sentence
    def build_child(self,sentence):
        dico={}
        sentence_c=sentence[1:-1]
        name_node=sentence_c.split()[0]
        dico[name_node]={}
        u=0
        terminal=(sentence_c.find("(")==-1)
        if(not terminal):
            sentence_c=sentence_c[sentence_c.find("("):]

            while(len(sentence_c.strip())>0):
                i=0
                find_open=False
                while find_open==False:
                    if sentence_c[i]=="(":
                        find_open=True
                        start=i
                        score=0
                        j=i
                        find_close_one=False
                        while find_close_one==False:
                            if sentence_c[j]=="(":
                                score+=-1
                            if sentence_c[j]==")":
                                score+=1
                            if score==0:
                                end=j
                                find_close_one=True
                            j+=1
                    i+=1

                element_to_remove="("+sentence_c[start:(end+1)][1:-1]+")"
                sentence_c=sentence_c[0:start]+sentence_c[(end+1):]
                key=str(u)
                u=u+1
                dico[name_node][name_node+"_"+key]=self.build_child(element_to_remove)

        else:

            v=sentence.split()
            sentence=" ".join(v) 
            dico=sentence 
            return(dico)

        return(dico)

    #add rules on a grammar from the previous tree
    def find_child(self,dico,grammar_tree):

        for k in dico.keys():
            node=k
        if node not in grammar_tree.keys():
            grammar_tree[node]=[]

        list_son=[]
        for k, value in dico[node].items():

            if type(value) == dict:
                for kb in value.keys():
                    son=kb
                self.find_child(dico[node][k],grammar_tree)
            else:
                son=value.split()[0][1:]

            list_son.append(son)

        grammar_tree[node].append("_".join(list_son))
       
    #build pcfg, add all rules to grammar and normalize to obtain probabilities
    def build_pcfg(self):

        for sentence in self.corpus:

            sentence=sentence[2:(len(sentence)-1)]
            self.find_child(self.build_child(sentence),self.grammar)

        for k,v in self.grammar.items():
            self.grammar[k]=dict(Counter(v))


        for k in self.grammar.keys():
            count=self.grammar[k]
            total = float(sum(count.values()))
            for kb in count:
                count[kb] /= total
            self.grammar[k]=dict(count)
    
    #create a matrix which entry detailed all the rules in order to gain time in CYK
    def build_mat_grammar(self):
        
        for k,v in self.grammar.items():
            for kb, vb in v.items():
        
                if len(kb.split("_"))==2:
                    isunary=False
                    #lhs, lrs, prob,lrs1,lrs2,isunary
                    self.mat_grammar.append([k,kb,vb,kb.split("_")[0],kb.split("_")[1],isunary])
                else:
                    isunary=True
                    self.mat_grammar.append([k,kb,vb,kb.split("_")[0],None,isunary])

     ###################Chomsky Normal Form########################
    
    #Find terminal node from the grammar in order to compute the chomsky normal form
    def find_term_node(self):
        term_node=[]
        for k,v in self.lexic.items():
            for kb in v.keys():
                term_node.append(kb)
        self.term_node=set(term_node) 
   
    
    #Eliminate non-solitary terminals
    def non_sol_term(self):
        chomsky_tree=copy.deepcopy(self.grammar)
        for k,v in self.grammar.items():
            for kb, vb in v.items():
                tmp=kb.split("_") 
                if(len(tmp)>1): 
                    for i,l in enumerate(tmp): 
                        if l in self.term_node:
                            tmp[i]="*"+l 
                            chomsky_tree["*"+l]={l:1.0} 

                    del chomsky_tree[k][kb] 
                    chomsky_tree[k]["_".join(tmp)]=vb

        self.grammar=chomsky_tree



    #Binarization
    def binarize(self):
        chomsky_tree=copy.deepcopy(self.grammar)
        for k,v in self.grammar.items():
            for kb, vb in v.items():
                tmp=kb.split("_")
                nb_split=len(tmp)-1
                if(nb_split>1): #if not we may just face an unary node problem for later
                    elt=tmp[0]
                    #split successively
                    for i in range(nb_split):

                        new_key="|".join(tmp[i+1:])
                        if i !=0:
                            chomsky_tree[old_key]={elt+"_"+new_key : 1.0} 
                        else:
                            chomsky_tree[k][elt+"_"+new_key]=vb
                        old_key=new_key
                        elt=tmp[i+1]

                    del chomsky_tree[k][kb]
        self.grammar=chomsky_tree
                    

    
    #Eliminate unary rules with non terminals POS
    def unary_rule(self):
        chomsky_tree=copy.deepcopy(self.grammar) 
     
        for k,v in self.grammar.items():
            #tous les noeuds possibles
            for kb in v.keys(): 
                if (len(kb.split("_"))==1 and kb not in self.term_node):# the rule k->kb  is unary
                        prob_trans=chomsky_tree[k][kb] # retain P: A-> B

                        del chomsky_tree[k][kb] 
                        for kt in self.grammar[kb].keys(): 

                            if chomsky_tree[k].get(kt) != None:
                                chomsky_tree[k][kt]+=prob_trans*self.grammar[kb][kt]  
                            else:
                                chomsky_tree[k][kt]=prob_trans*self.grammar[kb][kt] 
                                
        self.grammar=chomsky_tree
    
    #Erase unary non terminal rules remaining after the first unarization and renormalize (avoid cycles)
    def erase_last_unary(self):
        chomsky_tree=copy.deepcopy(self.grammar) 
        for k, v in self.grammar.items():
            normal=False
            for kb in v.keys(): 
                if len(kb.split("_"))==1 and kb not in self.term_node:
                    del chomsky_tree[k][kb]
                    normal=True
            
            #Renormalization of child of left hand rule k
            if normal:
                count=chomsky_tree[k]
                total = float(sum(count.values()))
                for kb in count:
                    count[kb] /= total
                    chomsky_tree[k]=dict(count)
       
        self.grammar=chomsky_tree
    
    

    ###################CYK Algorithm########################
   
    #Implementation of CYK algorithm with tree probabilities in order to keep the higher probability tree
    #Sentence with word in train in entry
    def PCYK(self,corrected_sentence):
        
        n_nodes=len(list(self.dic_nodes.keys()))
        n=len(corrected_sentence)
        prob = np.zeros((n,n,n_nodes))
        back = np.zeros((n,n,n_nodes),dtype=np.ndarray)


        for u,word in enumerate(corrected_sentence):
            for pos_lexic in self.lexic[word]:
                v = self.dic_nodes[pos_lexic]
                prob[0,u,v] = self.lexic[word][pos_lexic]
    
        #Rules which lead to terminals are unary
        for u in range(n):
            for line_rules in self.mat_grammar:
                    lhs, lrs, pr, lrs1,lrs2,isunary = line_rules
                    if isunary: 
                        pos_l = self.dic_nodes[lhs]
                        pos_r = self.dic_nodes[lrs]
                        prob_unary = prob[0,u,pos_r] * pr 
                        if prob[0,u,pos_r]>0 and prob_unary > prob[0,u,pos_l]: 
                            prob[0,u,pos_l] = prob_unary
                            back[0,u,pos_l] = (0,pos_r,-1)
        
        #Binary rules with 2 non terminals
        for l in progressbar.progressbar(range(2,n+1)):#length of sub sequence
            for u in range(1,n-l+2): #beginning of sub sequence
                for w in range(0,l): #where do we split
                        for line_rules in self.mat_grammar:
                            lhs, lrs, pr, lrs1,lrs2,isunary = line_rules
                            if not isunary:
                                pos_par = self.dic_nodes[lhs]
                                left = self.dic_nodes[lrs1]
                                right = self.dic_nodes[lrs2]
                                
                                prob_prod = pr
                                prob_binary = prob_prod * prob[w-1,u-1,left] * prob[l-w-1,u+w-1,right]
                               
                                if prob[w-1,u-1,left] > 0 and prob[l-w-1,u+w-1,right] > 0 and prob[l-1,u-1,pos_par] < prob_binary:
                                    
                                    prob[l-1,u-1,pos_par] = prob_binary
                                    back[l-1,u-1,pos_par] = (w,left,right)
            
        return(back, prob)
          
                            
     

    # Return the bracket sentence from the CYK back table   
    def build_bracket_sent(self,start_sent,back,l,start,pos,S=''): 
       
        indexes = back[l,start,pos]
        w,left,right = indexes
        if  left!=0 and right == -1 : 
            return "(" + list(self.dic_nodes.keys())[left]+ " " + start_sent[start] + ")"
       
        S += "(" +list(self.dic_nodes.keys())[left]+" "+ self.build_bracket_sent(start_sent,back,w-1,start,left) +")"
        S += "(" +list(self.dic_nodes.keys())[right]+" "+ self.build_bracket_sent(start_sent,back,l-w,start+w,right) +")" 

        return S
    
    #unchomsky a bracket sentence obtained from the build_bracket_sent function
    def unchomsky_bracket_sentence(self,res_cyk):
        while (res_cyk.find("(*")!= -1 or res_cyk.find("|")!=-1):

            if res_cyk.find("(*")== -1:
                st=res_cyk.find("|")

                for j,u in enumerate(reversed(res_cyk[:st])):
                    if u =="(":
                        break
                start=st-(j+1)

            else:
                start=res_cyk.find("(*")
            part=res_cyk[start:]
            #ok find the closing parenthesis
            score=0
            find_close_one=False
            i=-1
            while not find_close_one:
                #print(i)
                i=i+1
                if (part[i]=="("):
                    score+=1
                if (part[i]==")"):
                    score-=1
                if score==0:
                    find_close_one=True
                    end=start+i
                    res_cyk=res_cyk[:start]+" ".join(res_cyk[start:(end+1)].split()[1:])[:-1]+res_cyk[end+1:]
            
        return(res_cyk)

      
            
    


# Class OOV

In [12]:
class OOV: 
    
    def __init__(self,lexic_train,path_embeddings,unigram,bigram):
        self.lexic = lexic_train
        self.bigram = bigram
        self.unigram = unigram
        words, embeddings = pickle.load(open(path_embeddings, 'rb'),encoding='latin-1')
        self.dic_embedding = {k:v for k,v in zip(words,embeddings)}
    
    #Damerau-Levenshtein distance
    def dist_DL(self,w1,w2):
        n1 = len(w1)
        n2 = len(w2)
        m=np.zeros((n1+1,n2+1))

        for i  in range(n1+1):
            m[i, 0] = i
        for j in range(n2+1):
            m[0, j] = j
        for i in range(1,(n1+1)):
            for j in range(1,(n2+1)):
                
                if w1[i-1] == w2[j-1]:
                    cost = 0
                else:
                    cost = 1

                m[i,j] = min(m[i-1,j] + 1,m[i,j-1] + 1,m[i-1,j-1] + cost)

                if( w1[i-1] == w2[j-2] and w1[i-2] == w2[j-1]): #Transposition
                    m[i,j] = min(m[i,j],m[i-2,j-2] + cost) 

        return(m[n1,n2])

    #Cosine similarity
    def dist_cosine(self,w1,w2):
        a = self.dic_embedding[w1]
        b = self.dic_embedding[w2]
        return (np.dot(a,b)/(np.linalg.norm(a)*np.linalg.norm(b)))

    #Return n nearest neighbour computed with cosine similarity or DL 
    def nearest_neighbour(self,word,n,dist="dist_cosine"):
       
        if dist=="dist_cosine":
            distances = {e:self.dist_cosine(word,e) for e in self.lexic.keys() if e in self.dic_embedding.keys() }
            sorted_distances = sorted(distances.items(), key=operator.itemgetter(1))[::-1]
        else:
            distances = {e:self.dist_DL(word,e) for e in self.lexic.keys()}
            sorted_distances = sorted(distances.items(), key=operator.itemgetter(1))   
  
        return [s[0] for s in sorted_distances[:n]]
    
    #Return n_cosine nearest neighbour from dist cosine and n_lev nearest neighbour from DL distance
    def give_candidates_train(self,word,n_cosine,n_lev):
    
        list_cosine=[]
        if word in self.dic_embedding.keys():

            list_cosine=self.nearest_neighbour(word,n_cosine,dist="dist_cosine")

        list_levenstein=self.nearest_neighbour(word,n_lev,dist="Lev")

        return(list_cosine + list_levenstein) 
    
    #Process the whole sentence usingcandidates from the distances, unigram and bigram
    #Return a "word-in-train" sentence in order to feed PCYK algorithm
    def process_a_sentence(self,sent,n_cos,n_lev,hyper_lambda):
        sent_in_train=[]
        for i,word in enumerate(sent):
            if word in self.lexic.keys():
                sent_in_train.append(word) 
            else:  
                pmax=0
                for candidate in self.give_candidates_train(word,n_cos,n_lev):
                   
                    if i==0:
                        start="#s"
                    else:
                        start=sent_in_train[i-1] 

                    if oov.bigram[start].get(candidate)!= None:  
                        p= hyper_lambda * self.unigram[candidate]+(1-hyper_lambda)*self.bigram[start][candidate]
                    else:
                        p= hyper_lambda * self.unigram[candidate]

                    if p>pmax:
                        pmax=p
                        winner=candidate

                sent_in_train.append(winner)
       
        return(sent_in_train)




# Grammar initialization

In [21]:
train=CorpusManager(corpus_train)
train.erase_dash()
#print(train.corpus[3])
train.build_pcfg()
#pprint(train.grammar)
train.build_lexic()

In [22]:
train.lexic

{'Gutenberg': {'NPP': 1.0},
 'Cette': {'DET': 1.0},
 'exposition': {'NC': 1.0},
 'nous': {'CLO': 0.11450381679389313,
  'CLS': 0.8015267175572519,
  'CLR': 0.06870229007633588,
  'PRO': 0.015267175572519083},
 'apprend': {'V': 1.0},
 'que': {'CS': 0.7723342939481268,
  'PROREL': 0.1786743515850144,
  'ADV': 0.04899135446685879},
 'dès': {'P': 1.0},
 'le': {'DET': 0.9590062111801242, 'CLO': 0.040993788819875775},
 'XIIe': {'ADJ': 1.0},
 'siècle': {'NC': 1.0},
 ',': {'PONCT': 1.0},
 'à': {'P': 1.0},
 'Dammarie-sur-Saulx': {'NPP': 1.0},
 'entre': {'P': 1.0},
 'autres': {'ADJ': 0.972972972972973, 'PRO': 0.02702702702702703},
 'sites': {'NC': 1.0},
 'une': {'DET': 0.989010989010989, 'PRO': 0.01098901098901099},
 'industrie': {'NC': 1.0},
 'métallurgique': {'ADJ': 1.0},
 'existait': {'V': 1.0},
 '.': {'PONCT': 1.0},
 'à_peu_près': {'ADV': 1.0},
 'au': {'P+D': 1.0},
 'même': {'ADJ': 0.6774193548387096, 'ADV': 0.3225806451612903},
 'moment': {'NC': 1.0},
 'inventait': {'V': 1.0},
 "l'": {'DET'

In [5]:
#Grammar
print("learning corpus grammar")
train=CorpusManager(corpus_train)
train.erase_dash()
train.build_pcfg()
train.build_lexic()
train.find_term_node()

print("Put in chomsky normal form")
train.non_sol_term()
train.binarize()
train.unary_rule()
train.erase_last_unary()
train.build_mat_grammar()

train.build_nodes()
train.build_sentences()
train.build_bigram()
train.build_unigram()

#OOV
print("building OOV from corpus")
oov=OOV(lexic_train=train.lexic,path_embeddings='polyglot-fr.pkl',unigram=train.unigram,bigram=train.bigram)

print("finish")
val=CorpusManager(corpus_dev)
test=CorpusManager(corpus_test)
val.erase_dash()
test.erase_dash()
val.build_sentences()
test.build_sentences()

learning corpus grammar
Put in chomsky normal form
building OOV from corpus
finish


# Result From a text file 

In [6]:

corpus_to_file=[" ".join(x) for x in test.sentences if len(x)<=20]
target_bracket_sent=[x for x,y in zip(test.corpus,test.sentences) if len(y)<=20]


input_path = 'input.txt'
with open('input.txt','w') as f:
       for line in corpus_to_file:
            f.write(line)
            f.write("\n")

with open('input.txt','r') as f:
    lines = [line.strip('\n') for line in f]

sentence_to_parse=[]
for line in lines:
    sentence_to_parse.append(line.split())

sentence_to_parse[0]
target_bracket_sent[0]


pred_sent=[]
for i,sent in enumerate(sentence_to_parse):
        print(i+1,"/",len(sentence_to_parse))
        start_sent=sentence_to_parse[i]
 
        corrected_sent=oov.process_a_sentence(start_sent,2,2,0.2)
        back, prob=train.PCYK(corrected_sent)
        sol=train.build_bracket_sent(start_sent,back,len(corrected_sent)-1,0,train.dic_nodes["SENT"]) #add start sent here
        
        print("CYK")
        res_cyk="( (SENT "+sol+"))"
        pred_sent.append(train.unchomsky_bracket_sentence(res_cyk))
        print(train.unchomsky_bracket_sentence(res_cyk))

#np.save("pred_inf20.npy",pred_small_sent)
#np.save("target_inf20.npy",target_bracket_sent)

 25% (1 of 4) |######                    | Elapsed Time: 0:00:00 ETA:   0:00:00

1 / 174


100% (4 of 4) |##########################| Elapsed Time: 0:00:00 Time:  0:00:00


CYK
( (SENT (DET Le)(NC Procès)(PP (P en)(NP (ADJ première)(NC instance)))))
2 / 174


 25% (4 of 16) |######                   | Elapsed Time: 0:00:02 ETA:   0:00:09

KeyboardInterrupt: 

# Grid search on validation

In [ ]:
np.random.seed(2367)
l=list(np.arange(90))
np.random.shuffle(l)
#Only small one to test..
sentence_to_parse=[x for x in val.sentences if len(x)<=20]
target_bracket_sent=[x for x,y in zip(val.corpus,val.sentences) if len(y)<=20]

sentence_to_parse=[sentence_to_parse[i] for i in l]
target_bracket_sent=[target_bracket_sent[i] for i in l]

for hyper_lambd in [0.05,0.2,0.5,0.8,0.95]:

    pred_small_val_sent=[]
    for i,sent in enumerate(sentence_to_parse):
            print(i+1,"/",len(sentence_to_parse))
            start_sent_bracket=target_bracket_sent[i]
            start_sent=sentence_to_parse[i]

            corrected_sent=oov.process_a_sentence(start_sent,20,20,hyper_lambd)
            back, prob=train.PCYK(corrected_sent)
            sol=train.build_bracket_sent(start_sent,back,len(corrected_sent)-1,0,train.dic_nodes["SENT"]) #add start sent here

            res_cyk="( (SENT "+sol+"))"
            pred_small_val_sent.append(train.unchomsky_bracket_sentence(res_cyk))



    np.save("pred_small_20_20_"+str(hyper_lambd)+".npy",pred_small_val_sent)
    np.save("target_small_val.npy",target_bracket_sent)

N/A% (0 of 10) |                         | Elapsed Time: 0:00:00 ETA:  --:--:--

1 / 90


 40% (4 of 10) |##########               | Elapsed Time: 0:00:01 ETA:   0:00:02

# Scoring/ Tuning

In [10]:
path_pred="pred_small_4_4_"+str(hyper_lambd)+".npy"
len(np.load(path_pred))


50

In [11]:
#ok pad the bracket sentence and the tardet sentence 
#write the result on a text file without the beginning of the bracket and eval with gold etc..
for hyper_lambd in [0.05,0.2,0.5,0.8,0.95]:
    path_pred="pred_small_4_4_"+str(hyper_lambd)+".npy"
    path_target="target_small_val.npy"

    target_bracket=np.load(path_target)
    pred_bracket=np.load(path_pred)

    target_bracket=[x[2:][:-1] for x in target_bracket]
    pred_bracket=[x[2:][:-1] for x in pred_bracket]

    gold_path='gold_corpus.txt'
    test_path='test_corpus.txt'

    with open(gold_path,'w') as f:
           for line in target_bracket:
                f.write(line)
                f.write("\n")

    with open(test_path,'w') as f:
           for line in pred_bracket:
                f.write(line)
                f.write("\n")

    result_path = "res_"+"pred_small_20_20_"+str(hyper_lambd)+".txt"

    score=scorer.Scorer()
    score.evalb(gold_path, test_path, result_path)

Length Unmatched !
gold sentence:18
test sentence:1
------------------------------
Length Unmatched !
gold sentence:18
test sentence:1
------------------------------
Length Unmatched !
gold sentence:18
test sentence:1
------------------------------
Length Unmatched !
gold sentence:18
test sentence:1
------------------------------
Length Unmatched !
gold sentence:18
test sentence:1
------------------------------


# Results final on the test part 

In [ ]:
sentence_to_parse=test.sentences
target_bracket_sent=test.corpus

pred_all_sent=[]
for i,sent in enumerate(sentence_to_parse):
        print(i+1,"/",len(sentence_to_parse))
        start_sent_bracket=target_bracket_sent[i]
        start_sent=sentence_to_parse[i]

       
        corrected_sent=oov.process_a_sentence(start_sent,2,2,0.2)
        back, prob=train.PCYK(corrected_sent)
        sol=train.build_tree(start_sent,back,len(corrected_sent)-1,0,train.dic_nodes["SENT"]) #add start sent here
  
        res_cyk="( (SENT "+sol+"))"
        pred_all_sent.append(train.unchomsky(res_cyk))
        

np.save("pred_all.npy",pred_all_sent)
np.save("target_all.npy",target_bracket_sent)

# Test CYK

In [ ]:
def PCYK2(self,corrected_sentence):
    
        n_nodes=len(list(self.dic_nodes.keys()))
        n=len(corrected_sentence)
        prob = np.zeros((n,n,n_nodes))
        back = np.zeros((n,n,n_nodes),dtype=np.ndarray)


        for s,word in enumerate(corrected_sentence):
            for pos in self.lexic[word]:
                v = self.dic_nodes[pos]
                prob[0,s,v] = self.lexic[word][pos]
    
    #browse unaries
        for s in range(n):
            for l_h_s, prod in self.grammar.items():
                for kb,vb in prod.items():
                    r_h_s=kb.split("_")
                    if len(r_h_s)==1:
                        v1 = self.dic_nodes[l_h_s]
                        v2 = self.dic_nodes[r_h_s[0]]
                        prob_splitting = prob[0,s,v2] * vb #proba du lexic *proba que ça arrive
                        if prob[0,s,v2]>0 and prob_splitting > prob[0,s,v1]: 
                            prob[0,s,v1] = prob_splitting
                            back[0,s,v1] = (0,v2,-1)

        for l in progressbar.progressbar(range(2,n+1)): #length
            for s in range(1,n-l+2): #start
                for p in range(0,l): #cut cursor
                    for l_h_s, r_h_s in self.grammar.items(): 
                        a = self.dic_nodes[l_h_s]
                        
                        for kp,vp in r_h_s.items():    
                            sp=kp.split("_") 
                            if len(sp)==2:
                                
                                b = self.dic_nodes[sp[0]]
                                c = self.dic_nodes[sp[1]]
                                
                                prob_prod = vp
                                prob_splitting = prob_prod * prob[p-1,s-1,b] * prob[l-p-1,s+p-1,c]
                               
                                if prob[p-1,s-1,b] > 0 and prob[l-p-1,s+p-1,c] > 0 and prob[l-1,s-1,a] < prob_splitting:
                                    
                                    #print(l_h_s,"->",sp[0],sp[1])
                                    prob[l-1,s-1,a] = prob_splitting
                                    back[l-1,s-1,a] = (p,b,c)
            
        return(back, prob)

In [ ]:
chomsky_tree={}

dic1={"S":{ "A_B":1.0,"h":1.0}}
dic2={"A":{"C_C":0.1, "a":0.7,"c":0.2}}
dic3={"B":{"B_C":0.8,"b":0.2}}
dic4={"C":{"C_B":0.7,"B_A":0.2,"c":0.1}}

chomsky_tree.update(dic1)
chomsky_tree.update(dic2)
chomsky_tree.update(dic3)
chomsky_tree.update(dic4)
chomsky_tree

lexic={"banane":{"a":0.8,"c":0.2} ,"chocolat":{"b":0.1, "c":0.1 ,"h":0.8},"bleu":{"b":1}}
sent=["chocolat","chocolat","bleu","chocolat"]

#min imal examples, shpuld work as it work in the other notebook
ctest=CorpusManager(corpus_train)
ctest.grammar=chomsky_tree
ctest.lexic=lexic
ctest.build_nodes()

sent=["chocolat","chocolat","bleu","chocolat"]
back, prob =ctest.PCYK2(sent) # pour le test CYK 2

sol=ctest.build_tree(sent,sent,back,len(sent)-1,0,ctest.dic_nodes["S"]) #add start sent here
sol    
#back, prob =PCYK(test,sent)
##ok ça marche !!!
##le CYK MARCHE SUR UN ARBRE BIEN CONSTRUIT

In [ ]:
 def build_tree(self,start_sent,back,l,start,POS,S=''): 
       
        indexes = back[l,start,POS]
        if indexes == 0: 
            return start_sent[start]
        else: 
            p,left,right = indexes
            if  left!=0 and right == -1 : 
                return "(" + list(self.dic_nodes.keys())[left]+ " " + start_sent[start] + ")"
            #print(non_terminals[pos],non_terminals[b],non_terminals[c])
            S += "(" +list(self.dic_nodes.keys())[left]+" "+ self.build_tree(start_sent,back,p-1,start,left) +")"
            S += "(" +list(self.dic_nodes.keys())[right]+" "+ self.build_tree(start_sent,back,l-p,start+p,right) +")" 

        return S

